### Functions to calculate the circulations for temp and salinity

In [1]:
import xarray as xr
import numpy as np
import math
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt
import numpy.matlib

In [2]:
path = '/g/data3/hh5/tmp/access-om/fbd581/OUTPUT/1deg_yatm_jra55_ryf8485_fafctrl_norestore_passiveheat/output051/ocean/'

In [3]:
ds = xr.open_mfdataset(path + 'ocean.nc')
ds_area = xr.open_mfdataset(path + 'ocean_grid.nc')

/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/xarray/coding/times.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)


In [4]:
st_ocean = ds.st_ocean
time = ds.time
area_t = ds_area.area_t

In [5]:
temp_advection = ds.temp_advection * area_t
temp_vdiffuse_impl = ds.temp_vdiffuse_impl* area_t
temp_vdiffuse_diff_cbt_conv = ds.temp_vdiffuse_diff_cbt_conv* area_t
temp_nonlocal_kpp = ds.temp_nonlocal_KPP* area_t
neutral_gm_temp = ds.neutral_gm_temp* area_t
neutral_diffusion_temp = ds.neutral_diffusion_temp* area_t
temp_submeso =ds.temp_submeso* area_t
mixdownslope_temp = ds.mixdownslope_temp* area_t
sw_heat = ds.sw_heat* area_t
sfc_hflux_pme = ds.sfc_hflux_pme* area_t
temp_rivermix = ds.temp_rivermix* area_t
frazil_3d = ds.frazil_3d* area_t
temp_tendency = ds.temp_tendency* area_t
temp_vdiffuse_sbc = ds.temp_vdiffuse_sbc* area_t

temp = ds.temp-273.15

In [6]:
ADV_temp = temp_advection                                                     # % Advection;                                   % UNit = W/m^2
DIA_temp = temp_vdiffuse_impl - temp_vdiffuse_diff_cbt_conv                     # % Dianeutral diffusion or vertical dissusion;  % UNit = W/m^2
KPP_temp = temp_nonlocal_kpp                                                  # % non-local KPP;                               % UNit = W/m^2
EIT_temp = neutral_gm_temp + neutral_diffusion_temp                             # % Eddy-induced transport;                      % Unit =W/m^2
EIT_ADV_temp = neutral_gm_temp                                                # % Eddy-induced advection transport;            % Unit =W/m^2
SUB_temp = temp_submeso                                                       # % Submesoscale eddies;                         % UNit = W/m^2
CON_temp = temp_vdiffuse_diff_cbt_conv+mixdownslope_temp                      # % Convection;                                  % UNit = W/m^2
SWP_temp = sw_heat                                                            # % Shortwave penetration;                       % Unit = W/m^2
PME_temp = sfc_hflux_pme                                                      # % Precipitation minis evaporation;             % Unit =W/m^2
RIV_temp = temp_rivermix                                                      # % River runnoff;                               % UNit = W/m^2
FRZ_temp = frazil_3d                                                          # % Frazil;                                      % Unit = W/m^2
NET_temp = temp_tendency                                                      # % Frazil;                                      % Unit = W/m^2

In [10]:
def diagnostic_cal(var, t_s, t_s_range):
          
    var2_tempz_bin_temp = np.zeros((np.size(var,0), np.size(t_s_range), np.size(var,1)))
    
    for i in range(np.size(var,0)):
        
        for ii in range(np.size(var,1)):
                                   
            Tonz = np.squeeze(t_s[i, ii, :, :]) 
            ind = np.digitize(np.ravel(Tonz), t_s_range)
            
            var_tempz = np.squeeze(var[i, ii, :, :])
            totfre1 = np.bincount(np.ravel(ind), weights = np.ravel(var_tempz))
            var2_tempz_bin_temp[i, :, ii] = totfre1[1:]
            
        var2_bin = var2_tempz_bin_temp
        
    return var2_bin


In [12]:
delta_salt = 0.05
delta_temp = 0.5
rho = 1035
cp = 3992

# Why?:
x_salt = delta_salt
x_temp = rho * cp * delta_temp

dz = st_ocean
temp_range = np.arange(-3, 35, delta_temp)
salt_range =  np.arange(0, 45, delta_salt)

In [ ]:
adv_bin = diagnostic_cal(ADV_temp, temp, temp_range)

In [24]:
def circulation_cal(var, x):
    
    ADV_mean_mean = np.nanmean(var, axis = 0)
    ADV_mean_new = np.squeeze(ADV_mean_mean)
    cumdPSi_temp_mean = (np.cumsum(ADV_mean_new, 1))

    cumdPSi_temp_mean_dz = np.transpose(np.matlib.repmat(cumdPSi_temp_mean[:,-1], np.size(var, 2), 1)) - cumdPSi_temp_mean
    cumdPSi_temp_mean_m3 = cumdPSi_temp_mean_dz / x
    cumdPSi_temp_mean_SV = cumdPSi_temp_mean_m3 * 1e-6
    
    return cumdPSi_temp_mean_SV

In [25]:
test = circulation_cal(ADV_temp, x_temp)

/jobfs/37238249.gadi-pbs/ipykernel_4155825/883261819.py:3: RuntimeWarning: Mean of empty slice
  ADV_mean_mean = np.nanmean(var, axis = 0)


ValueError: operands could not be broadcast together with shapes (360,15000) (50,300,360) 